#Домашнее задание 3:
- другие варианты ранжированивания айтемов похожих пользователей (2 балла)

In [1]:
!pip install rectools==0.2.0 implicit >> None

In [2]:
import pandas as pd
import numpy as np
import scipy as sp
import requests
from tqdm.auto import tqdm
from scipy.stats import mode 
from pprint import pprint
from implicit.nearest_neighbours import CosineRecommender, TFIDFRecommender, ItemItemRecommender, BM25Recommender
import warnings
warnings.filterwarnings("ignore")

from rectools import Columns

pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', 200)

# 🎬 Get KION dataset 

<a href="https://ods.ai/competitions/competition-recsys-21/data"> Dataset description [ru] </a>


In [4]:
# download dataset by chunks
url = "https://storage.yandexcloud.net/itmo-recsys-public-data/kion_train.zip"

req = requests.get(url, stream=True)

with open('kion_train.zip', "wb") as fd:
    total_size_in_bytes = int(req.headers.get('Content-Length', 0))
    progress_bar = tqdm(desc='kion dataset download', total=total_size_in_bytes, unit='iB', unit_scale=True)
    for chunk in req.iter_content(chunk_size=2 ** 20):
        progress_bar.update(len(chunk))
        fd.write(chunk)

kion dataset download:   0%|          | 0.00/78.8M [00:00<?, ?iB/s]

In [5]:
!unzip kion_train.zip

Archive:  kion_train.zip
   creating: kion_train/
  inflating: kion_train/interactions.csv  
  inflating: __MACOSX/kion_train/._interactions.csv  
  inflating: kion_train/users.csv    
  inflating: __MACOSX/kion_train/._users.csv  
  inflating: kion_train/items.csv    
  inflating: __MACOSX/kion_train/._items.csv  


# EDA

In [3]:
interactions = pd.read_csv('kion_train/interactions.csv')
users = pd.read_csv('kion_train/users.csv')
items = pd.read_csv('kion_train/items.csv')

In [4]:
# rename columns, convert timestamp
interactions.rename(columns={'last_watch_dt': Columns.Datetime,
                            'total_dur': Columns.Weight}, 
                    inplace=True) 

interactions['datetime'] = pd.to_datetime(interactions['datetime'])

## interactions

In [5]:
pd.concat([interactions.head(), interactions.tail()])

,user_id,item_id,datetime,weight,watched_pct
0,176549,9506,2021-05-11,4250,72.0
1,699317,1659,2021-05-29,8317,100.0
2,656683,7107,2021-05-09,10,0.0
3,864613,7638,2021-07-05,14483,100.0
4,964868,9506,2021-04-30,6725,100.0
5476246,648596,12225,2021-08-13,76,0.0
5476247,546862,9673,2021-04-13,2308,49.0
5476248,697262,15297,2021-08-20,18307,63.0
5476249,384202,16197,2021-04-19,6203,100.0
5476250,319709,4436,2021-08-15,3921,45.0


In [6]:
print(f"Interactions dataframe shape: {interactions.shape}")
print(f"Unique users in interactions: {interactions['user_id'].nunique():_}")
print(f"Unique items in interactions: {interactions['item_id'].nunique():_}")

Interactions dataframe shape: (5476251, 5)
Unique users in interactions: 962_179
Unique items in interactions: 15_706


In [7]:
max_date = interactions['datetime'].max()
min_date = interactions['datetime'].min()

print(f"min date in interactions: {min_date}")
print(f"max date in interactions: {max_date}")

min date in interactions: 2021-03-13 00:00:00
max date in interactions: 2021-08-22 00:00:00


In [8]:
interactions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5476251 entries, 0 to 5476250
Data columns (total 5 columns):
 #   Column       Dtype         
---  ------       -----         
 0   user_id      int64         
 1   item_id      int64         
 2   datetime     datetime64[ns]
 3   weight       int64         
 4   watched_pct  float64       
dtypes: datetime64[ns](1), float64(1), int64(3)
memory usage: 208.9 MB


## users

In [9]:
pd.concat([users.head(), users.tail()])

,user_id,age,income,sex,kids_flg
0,973171,age_25_34,income_60_90,М,1
1,962099,age_18_24,income_20_40,М,0
2,1047345,age_45_54,income_40_60,Ж,0
3,721985,age_45_54,income_20_40,Ж,0
4,704055,age_35_44,income_60_90,Ж,0
840192,339025,age_65_inf,income_0_20,Ж,0
840193,983617,age_18_24,income_20_40,Ж,1
840194,251008,NaN,NaN,NaN,0
840195,590706,NaN,NaN,Ж,0
840196,166555,age_65_inf,income_20_40,Ж,0


In [10]:
print(f"Users dataframe shape {users.shape}")
print(f"Unique users: {users['user_id'].nunique():_}")

Users dataframe shape (840197, 5)
Unique users: 840_197


## items

In [11]:
pd.concat([items.head(3), items.tail(3)])

,item_id,content_type,title,title_orig,release_year,genres,countries,for_kids,age_rating,studios,directors,actors,description,keywords
0,10711,film,Поговори с ней,Hable con ella,2002.0,"драмы, зарубежные, детективы, мелодрамы",Испания,NaN,16.0,NaN,Педро Альмодовар,"Адольфо Фернандес, Ана Фернандес, Дарио Грандинетти, Джеральдин Чаплин, Елена Анайя, Каэтано Велозо, Леонор Уотлинг, Лола Дуэньяс, Лолес Леон, Малу Айродо, Мариола Фуэнтес, Пас Вега, Пина Бауш, Ро...",Мелодрама легендарного Педро Альмодовара «Поговори с ней» в 2003 году получила премию «Оскар» за лучший сценарий. Журналист Марко берет интервью у знаменитой женщины-тореро Лидии и вскоре влюбляе...,"Поговори, ней, 2002, Испания, друзья, любовь, сильные, женщины, преодоление, трудностей, отношения, дружба, отношения, паре, отношения, мужчины, женщины, романтические, отношения, потеря, близких,..."
1,2508,film,Голые перцы,Search Party,2014.0,"зарубежные, приключения, комедии",США,NaN,16.0,NaN,Скот Армстронг,"Адам Палли, Брайан Хаски, Дж.Б. Смув, Джейсон Манцукас, Джон Глейсер, Карл Грин, Кристен Риттер, Лэнс Реддик, Морис Комт, Патрик Кернс, Ребекка Коллинз, Роза Салазар, Росс П. Кук, Стеффи Гроут, Ти...","Уморительная современная комедия на популярную тему о том, как не надо отмечать мальчишник. Главный герой усвоил, что не надо звать на свадьбу своего друга Джейсона, из-за которого он вместо сваде...","Голые, перцы, 2014, США, друзья, свадьбы, преодоление, трудностей, расставания, отношения, дружба, риск, недоразумение, мужская, дружба, мальчишники, девичники"
2,10716,film,Тактическая сила,Tactical Force,2011.0,"криминал, зарубежные, триллеры, боевики, комедии",Канада,NaN,16.0,NaN,Адам П. Калтраро,"Адриан Холмс, Даррен Шалави, Джерри Вассерман, Дэн Риззуто, Кендес Илэйн Калтраро, Кит Джардин, Лекса Дойг, Майкл Джей Уайт, Майкл Шэнкс, Майкл Эклунд, Питер Брайант, Питер Кент, Стив Бачич, Стив ...","Профессиональный рестлер Стив Остин («Все или ничего») и темнокожий мачо Майкл Джей Уайт («Темный рыцарь») в интригующем криминальном боевике. В центре сюжета – команда спецназовцев, которая оказа...","Тактическая, сила, 2011, Канада, бандиты, гангстеры, преступления, преодоление, трудностей, убийства, убийцы, настоящие, мужчины, риск, недоразумение, силы, правопорядка, борьба, за, выживание, сп..."
15960,10632,series,Сговор,Hassel,2017.0,"драмы, триллеры, криминал",Россия,0.0,18.0,NaN,"Эшреф Рейбрук, Амир Камдин, Эрик Эгер","Ола Рапас, Алиетт Офейм, Уильма Лиден, Шанти Рони, Тома Холмин","Криминальная драма по мотивам романов о шведском детективе Роланде Хасселе. Средь бела дня убит полицейский, и нити в этом деле ведут прямо в коридоры власти. Расследованием занимается детектив Ха...","Сговор, 2017, Россия"
15961,4538,series,Среди камней,Darklands,2019.0,"драмы, спорт, криминал",Россия,0.0,18.0,NaN,"Марк О’Коннор, Конор МакМахон","Дэйн Уайт О’Хара, Томас Кэйн-Бирн, Джудит Родди, Марк О’Халлоран, Джимми Смоллхорн","Семнадцатилетний Дэмиен мечтает вырваться за пределы своего района и стать профессиональным бойцом. Когда его кумир и старший брат исчезает, парень попадает в чуждый ему мир насилия, наркотиков и ...","Среди, камней, 2019, Россия"
15962,3206,series,Гоша,NaN,2019.0,комедии,Россия,0.0,16.0,NaN,Михаил Миронов,"Мкртыч Арзуманян, Виктория Рунцова","Добродушный Гоша не может выйти из дома, чтобы не попасть в нелепую и курьёзную историю. Но даже неудачники мечтают о любви, и наш герой — не исключение, ведь оптимизма ему не занимать.","Гоша, 2019, Россия"


In [12]:
print(f"Items dataframe shape {items.shape}")
print(f"Unique item_id: {items['item_id'].nunique():_}")

Items dataframe shape (15963, 14)
Unique item_id: 15_963


# 🧩 New model: from `itemkNN` to `userkNN `

- we'll use `implicit.nearest_neighbours` itemKNN model and convert it to userkNN model 


## 0. train test split 

In [13]:
# train test split 
# test = last 1 week 
from rectools.model_selection import TimeRangeSplit

n_folds = 1
unit = "W"
n_units = 1
periods = n_folds + 1
freq = f"{n_units}{unit}"

last_date = interactions[Columns.Datetime].max().normalize()
start_date = last_date - pd.Timedelta(n_folds * n_units + 1, unit=unit)  
print(f"Start date and last date of the test fold: {start_date, last_date}")
    
date_range = pd.date_range(start=start_date, periods=periods, freq=freq, tz=last_date.tz)
print(f"Test fold borders: {date_range.values.astype('datetime64[D]')}")

# generator of folds
cv = TimeRangeSplit(
    date_range=date_range,
    filter_already_seen=True,
    filter_cold_items=True,
    filter_cold_users=True,
)
print(f"Real number of folds: {cv.get_n_splits(interactions)}")

Start date and last date of the test fold: (Timestamp('2021-08-08 00:00:00'), Timestamp('2021-08-22 00:00:00'))
Test fold borders: ['2021-08-08' '2021-08-15']
Real number of folds: 1


In [14]:
# we have just 1 test fold - no need to iterate over fold
(train_ids, test_ids, fold_info) = cv.split(interactions, collect_fold_stats=True).__next__()

In [15]:
train_ids

array([      0,       1,       2, ..., 5476245, 5476247, 5476249])

In [16]:
test_ids

array([      6,      33,      56, ..., 5476229, 5476230, 5476240])

# 1. Prepare train matrix 

### 1.1 Create `user` and `item` mappings (essential part for recsys models) 

users_mapping = `{user0: 0, user1: 1, ... , userN: N}`

items_mapping = `{item0: 0, item1: 1, ... , itemK: K}`

In [17]:
train = interactions.loc[train_ids]
test = interactions.loc[test_ids]

In [18]:
users_inv_mapping = dict(enumerate(train['user_id'].unique()))
users_mapping = {v: k for k, v in users_inv_mapping.items()}

In [19]:
items_inv_mapping = dict(enumerate(train['item_id'].unique()))
items_mapping = {v: k for k, v in items_inv_mapping.items()}

In [20]:
print(f"users_mapping amount: {len(users_mapping)}")
print(f"items_mapping amount: {len(items_mapping)}")

users_mapping amount: 842129
items_mapping amount: 15404


### 2.2 Get sparse matrix 

In [21]:
def get_coo_matrix(df, 
                   user_col='user_id', 
                   item_col='item_id', 
                   weight_col=None, 
                   users_mapping=None, 
                   items_mapping=None):
    if weight_col:
        weights = df[weight_col].astype(np.float32)
    else:
        weights = np.ones(len(df), dtype=np.float32)

    interaction_matrix = sp.sparse.coo_matrix((
        weights, 
        (
            df[user_col].map(users_mapping.get), 
            df[item_col].map(items_mapping.get)
        )
    ))
    return interaction_matrix

In [22]:
interaction_matrix = get_coo_matrix(train, users_mapping=users_mapping, items_mapping=items_mapping)

In [23]:
interaction_matrix

<842129x15404 sparse matrix of type '<class 'numpy.float32'>'
	with 4587708 stored elements in COOrdinate format>

## 2. Fit simple ItemKNN model 

### `userknn disclaimer`:

implicit ItemItemRecommender requires `item-user matrix` (not `user-item` !) => ususally you call fit with transposed weights matrix `model.fit(matrix.T)`. That's how you get item's nearest neighbours. 

but for `the userknn model` we want to have **user** neighbours (not item neighbours). that's why we need to call fit **without transpose**: `model.fit(matrix)`

https://github.com/benfred/implicit/blob/main/implicit/nearest_neighbours.py

In [24]:
userknn = CosineRecommender(K=30)
userknn.fit(interaction_matrix)

  0%|          | 0/842129 [00:00<?, ?it/s]

In [25]:
# save model
import dill

with open('userknn.dill', 'wb') as f:
    dill.dump(userknn, f)

In [ ]:
with open('userknn.dill', 'rb') as f:
    userknn = dill.load(f)

In [26]:
userknn.similar_items(1)

[(1, 0.9999999999999987),
 (78101, 0.28067570844923023),
 (273835, 0.2706231506959187),
 (496026, 0.26518576139191),
 (359238, 0.25514595333753737),
 (4656, 0.2546084638985143),
 (159253, 0.2544200743322844),
 (66832, 0.25157730271331386),
 (198831, 0.25157730271331386),
 (152958, 0.24751933820372524)]

## 3. Recommend to get user neighbours

In [27]:
def generate_implicit_recs_mapper(model, N, users_mapping, users_inv_mapping):
    def _recs_mapper(user):
        user_id = users_mapping[user]
        recs = model.similar_items(user_id, N=N)
        return [users_inv_mapping[user] for user, _ in recs], [sim for _, sim in recs]
    return _recs_mapper

In [28]:
mapper = generate_implicit_recs_mapper(
    userknn, 
    N=30,
    users_mapping=users_mapping,
    users_inv_mapping=users_inv_mapping
)

In [29]:
recs = pd.DataFrame({
    'user_id': test['user_id'].unique()
})

recs['similar_user_id'], recs['similarity'] = zip(*recs['user_id'].map(mapper))
recs.head()

,user_id,similar_user_id,similarity
0,1016458,"[1016458, 541677, 119761, 899134, 617870, 631868, 415485, 558286, 282770, 258857, 411860, 508723, 1038186, 659778, 325937, 365900, 28485, 66889, 19363, 331004, 245436, 1087135, 586282, 5248, 28706...","[0.9999999999999997, 0.3627381250550058, 0.3244428422615251, 0.3244428422615251, 0.3244428422615251, 0.3244428422615251, 0.32444284226152503, 0.30779350562554625, 0.3065696697424829, 0.29346959282..."
1,68478,"[68478, 76520, 1014710, 334639, 809149, 594095, 569771, 368275, 499111, 642773, 1048805, 430951, 737870, 230556, 128129, 392805, 966090, 390869, 647402, 286515, 320989, 866310, 503165, 584648, 488...","[0.9999999999999999, 0.4999999999999999, 0.4999999999999999, 0.408248290463863, 0.408248290463863, 0.408248290463863, 0.408248290463863, 0.408248290463863, 0.408248290463863, 0.408248290463863, 0...."
2,580093,"[580093, 1076677, 401821, 207688, 674145, 997821, 410517, 460268, 338204, 1023392, 739073, 817646, 545135, 295141, 237895, 1085503, 1084699, 553029, 72794, 338811, 393382, 816092, 96482, 279541, 4...","[1.0000000000000002, 0.6454972243679029, 0.6454972243679029, 0.6454972243679029, 0.6123724356957946, 0.5892556509887897, 0.5892556509887897, 0.5892556509887897, 0.5892556509887897, 0.5892556509887..."
3,1072552,"[1072552, 686410, 896269, 214005, 301548, 297202, 455232, 490740, 771482, 333766, 162216, 572395, 767106, 453394, 242419, 177232, 802122, 473056, 261159, 426951, 537688, 298131, 107758, 471676, 44...","[1.0000000000000007, 0.4277926319464987, 0.422200330920749, 0.4200840252084029, 0.407045789205347, 0.407045789205347, 0.4042260417272216, 0.4001633653325208, 0.4001633653325208, 0.4001633653325208..."
4,910002,"[910002, 849167, 1077278, 293899, 254697, 326828, 43033, 754229, 157889, 456303, 370936, 1038893, 244231, 399158, 366968, 81567, 1037560, 689838, 279642, 136003, 250111, 649259, 579048, 19373, 331...","[1.0000000000000007, 0.529150262212918, 0.48989794855663577, 0.4666666666666666, 0.4666666666666666, 0.4535573676110726, 0.4535573676110726, 0.4472135954999579, 0.4472135954999579, 0.4472135954999..."


In [30]:
# explode lists to get vertical representation
recs = recs.set_index('user_id').apply(pd.Series.explode).reset_index()

In [31]:
recs.head(30 + 5)

,user_id,similar_user_id,similarity
0,1016458,1016458,1.0
1,1016458,541677,0.362738
2,1016458,119761,0.324443
3,1016458,899134,0.324443
4,1016458,617870,0.324443
5,1016458,631868,0.324443
6,1016458,415485,0.324443
7,1016458,558286,0.307794
8,1016458,282770,0.30657
9,1016458,258857,0.29347


In [32]:
# delete recommendations of itself 
recs = recs[~(recs['similarity'] >= 1)]

In [33]:
recs.shape

(2583755, 3)

In [34]:
recs.head()

,user_id,similar_user_id,similarity
0,1016458,1016458,1.0
1,1016458,541677,0.362738
2,1016458,119761,0.324443
3,1016458,899134,0.324443
4,1016458,617870,0.324443


## 4. Join watched items of neighbour users to get item recommendations

In [35]:
watched = train.groupby('user_id').agg({'item_id': list})
watched.head()

,item_id
user_id,
0,"[7102, 14359, 15297, 6006, 9728, 12192]"
2,"[7571, 3541, 15266, 13867, 12841, 10770, 4475, 9506, 8936, 11018, 11577, 561, 7106, 6774, 16029, 8482, 6825, 3594, 16166, 5819, 2954, 383, 11689, 12449, 2025, 6155, 3628, 334, 4024, 7210, 11539, 1..."
3,"[9728, 16406, 10440, 3475, 4151, 1418, 2220, 3734, 13789, 8581, 13849, 9550, 10464, 4880, 4436, 8801, 11790, 2657, 8252]"
4,"[4700, 6317]"
5,"[14397, 6445, 11437, 5651, 6167, 12466, 632, 8450, 2685, 7825, 3145, 7043, 4179, 15890, 5115, 4719, 10848]"


In [36]:
# join watched items
recs = recs.merge(watched, left_on=['similar_user_id'], right_on=['user_id'], how='left')
recs = recs.explode('item_id')

In [37]:
recs.head()

,user_id,similar_user_id,similarity,item_id
0,1016458,1016458,1.0,1896
0,1016458,1016458,1.0,13865
0,1016458,1016458,1.0,4533
0,1016458,1016458,1.0,16152
0,1016458,1016458,1.0,3532


In [38]:
# drop duplicates pairs user_id-item_id 
# keep with the largest similiarity
recs = recs.sort_values(['user_id', 'similarity'], ascending=False)
recs.head()

,user_id,similar_user_id,similarity,item_id
2430429,1097544,1097544,1.0,12768
2430429,1097544,1097544,1.0,9381
2430429,1097544,1097544,1.0,12455
2430429,1097544,1097544,1.0,6309
2430429,1097544,1097544,1.0,9996


In [39]:
recs = recs.drop_duplicates(['user_id', 'item_id'], keep='first')

In [40]:
recs.shape

(3170125, 4)

## 5. Make `rank` from similarity



Добавим колонку с годом релиза и по ней будем ранжировать similarity

In [43]:
recs2 = recs.merge(items, how='left', on='item_id')

In [44]:
recs2

,user_id,similar_user_id,similarity,item_id,content_type,title,title_orig,release_year,genres,countries,for_kids,age_rating,studios,directors,actors,description,keywords
0,1097544,1097544,1.0,12768,film,Птицы,"Birds, The",1963.0,"драмы, ужасы, детективы, мелодрамы",США,NaN,12.0,NaN,Альфред Хичкок,"Род Тейлор, Джессика Тэнди, Сюзанн Плешетт, Типпи Хедрен, Вероника Картрайт, Этель Гриффиз, Чарльз МакГроу, Рут МакДевитт, Лонни Чэпмен, Джо Мэнтелл","Классический фильм А.Хичкока, в котором ужасающая картина нападения птиц на американский поселок переплетается с историей любовных взаимоотношений молодой женщины с понравившимся ей мужчиной.","зоомагазин, уединение, детская площадка, чайка, камин, нападение птиц, светская львица, по мотивам рассказа, розыгрыш, школьный учитель, неразлучник, лавочник, неразгаданная тайна, здание школы, 1..."
1,1097544,1097544,1.0,9381,film,Кто не спрятался,The Rental,2020.0,"драмы, ужасы, триллеры, детективы",США,NaN,18.0,NaN,Дэйв Франко,"Дэн Стивенс, Элисон Бри, Тоби Хасс, Шейла Ванд, Джереми Аллен Уайт, Конни Веллман, Чанк, Энтони Молинари, Эмити Бэйкон, Чейз Баркер",Вы снимали жилье через интернет? Смотрели красивые картинки на сайте и представляли идеальный отпуск? Две молодые пары решили провести выходные на берегу океана и арендовали роскошную виллу. Они н...,"камера, экстази, брат, дом, сталкер, убийство, вуайеризм, аренда, отпуск на выходных, 2020, соединенные штаты, кто, не, спрятался"
2,1097544,1097544,1.0,12455,series,Зебра в клеточку,Squared Zebra,2020.0,"мультсериалы, приключения",Россия,NaN,0.0,NaN,"Алексей Алексеев, Анна Борисова, Максим Куликов","Ольга Кузьмина, Владимир Паляница, Лариса Брохман, Диомид Виноградов, Мария Хамидуллина, Алексей Войтюк, Ольга Шорохова","Центральный персонаж – очень необычная зебра, Зебра в клеточку, которая родилась в джунглях у полосатых мамы и папы и которая, по всей видимости, даже не настоящая зебра, о чем ей в открытую говор...",NaN
3,1097544,1097544,1.0,6309,film,Карантин,Quarantine,2008.0,"ужасы, триллеры","США, Испания",NaN,18.0,NaN,Джон Эрик Даудл,"Дженнифер Карпентер, Стив Харрис, Джей Эрнандес, Джонатон Шек, Коламбус Шорт, Эндрю Фисцелла, Раде Шербеджия, Грег Джерманн, Бернард Уайт, Даня Рамирес","Телерепортёр Анжела Видал и её оператор делают репортаж о ночной смене лос-анджелесского пожарного управления. Рутинный вызов приводит их к зданию в центре города. Оказывается, что живущая здесь ж...","карантин, римейк, тележурналист, найденные кадры, вирус, 2008, соединенные штаты, испания"
4,1097544,1097544,1.0,9996,series,Немцы,Nemtsy,2021.0,драмы,Россия,NaN,16.0,NaN,Стас Иванов,"Евгений Коряковский, Анна Завтур, Антон Васильев, Дарья Урсуляк, Валерия Ланская, Юлия Марченко, Виталий Коваленко, Алексей Гришин, Владимир Устюгов, Александр Овчинников, Джульетта Геринг","На журналиста-оппозиционера Антона Эбергарда сваливается множество проблем — он теряет работу, терпит неудачу как писатель, влезает в долги. Антон отчаянно пытается найти деньги и в конце концов и...","немцы, немец, про немцев, по мотивам романа, провинциальная жизнь, провинция, взяточничество, коррупция, журналист, писатель, семейные отношения, экранизация, фильмы по книгам, немцы сериал, сериа..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3170120,3,248305,0.490511,760,film,Легенда № 17,Legenda No. 17,2012.0,"драмы, спорт, биография",Россия,NaN,6.0,NaN,Николай Лебедев,"Олег Меньшиков, Светлана Иванова, Владимир Меньшов, Роман Мадянов, Борис Щербаков, Даниэль Ольбрыхский, Нина Гребешкова, Данила Козловский, Александр Лобанов, Сергей Генкин, Александр Яковлев, Але...",2 сентября 1972 года. Монреаль. Хоккейная сборная СССР с разгромным счетом 7:3 победила канадских профессионалов из НХЛ в стартовом матче эпохальной Суперсерии СССР-Канада. Это была не просто игра...,"биография, спорт, хоккей на льду, Авария с участием грузовика, Автомобиль, Автомобильная авария, Балкон, Бег быков, Бык, Дядя, Испанец, Испания, Канада, Канадец, Квартира, Команда, Мать, Монреаль,.

In [45]:
recs2.columns

Index(['user_id', 'similar_user_id', 'similarity', 'item_id', 'content_type',
       'title', 'title_orig', 'release_year', 'genres', 'countries',
       'for_kids', 'age_rating', 'studios', 'directors', 'actors',
       'description', 'keywords'],
      dtype='object')

In [47]:
recs3=recs2.drop(['content_type',
       'title', 'title_orig', 'genres', 'countries',
       'for_kids', 'age_rating', 'studios', 'directors', 'actors',
       'description', 'keywords'], axis=1)

In [48]:
recs3

,user_id,similar_user_id,similarity,item_id,release_year
0,1097544,1097544,1.0,12768,1963.0
1,1097544,1097544,1.0,9381,2020.0
2,1097544,1097544,1.0,12455,2020.0
3,1097544,1097544,1.0,6309,2008.0
4,1097544,1097544,1.0,9996,2021.0
...,...,...,...,...,...
3170120,3,248305,0.490511,760,2012.0
3170121,3,248305,0.490511,16447,1999.0
3170122,3,248305,0.490511,12192,2019.0
3170123,3,248305,0.490511,11778,2009.0


In [50]:
recs3 = recs3.sort_values(['user_id', 'similarity', 'release_year'], ascending=False)
recs3.head()

,user_id,similar_user_id,similarity,item_id,release_year
4,1097544,1097544,1.0,9996,2021.0
6,1097544,1097544,1.0,4151,2021.0
7,1097544,1097544,1.0,7829,2021.0
1,1097544,1097544,1.0,9381,2020.0
2,1097544,1097544,1.0,12455,2020.0


In [52]:
# make rank
recs3['rank'] = recs3.groupby('user_id').cumcount() + 1 

In [53]:
recs3.head()

,user_id,similar_user_id,similarity,item_id,release_year,rank
4,1097544,1097544,1.0,9996,2021.0,1
6,1097544,1097544,1.0,4151,2021.0,2
7,1097544,1097544,1.0,7829,2021.0,3
1,1097544,1097544,1.0,9381,2020.0,4
2,1097544,1097544,1.0,12455,2020.0,5


## 6. Metrics

In [58]:
from rectools.metrics import MAP, Precision, Recall, MeanInvUserFreq, Serendipity, calc_metrics

# calculate several classic (precision@k and recall@k) and "beyond accuracy" metrics
metrics = {
    "MAP@10": MAP(k=10),
    "prec@10": Precision(k=10),
    "recall@10": Recall(k=10),
    "novelty": MeanInvUserFreq(k=10),
    "serendipity": Serendipity(k=10),
}

catalog = train['item_id'].unique()
    
metric_values = calc_metrics(
            metrics,
            reco=recs3,
            interactions=test,
            prev_interactions=train,
            catalog=catalog
        )

In [59]:
metric_values

{'prec@10': 0.00668248629673573,
 'recall@10': 0.03464901099883065,
 'MAP@10': 0.00784739276938693,
 'novelty': 6.505195815070313,
 'serendipity': 2.9280520069995032e-05}

метрики prec@10 и recall@10 получились выше, чем с использованием tfidf на семинаре, а novelty и serendipity ниже, можно использовать такое ранжирование в зависимости о задачи.